# Fundamentals of Pytorch

In [31]:
import torch

In [32]:
X = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)
X

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [33]:
X.shape

torch.Size([2, 3])

In [34]:
X.dtype

torch.float32

In [35]:
X[0, 2]

tensor(3.)

In [36]:
print(X.mean())
print(X.sum())
print(X.std())
print(X.abs())
print(X.argmax())
print(X.exp())
print(X.sqrt())

tensor(3.5000)
tensor(21.)
tensor(1.8708)
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor(5)
tensor([[  2.7183,   7.3891,  20.0855],
        [ 54.5981, 148.4132, 403.4288]])
tensor([[1.0000, 1.4142, 1.7321],
        [2.0000, 2.2361, 2.4495]])


In [37]:
10 * (X + 3)

tensor([[40., 50., 60.],
        [70., 80., 90.]])

In [38]:
X.T @ X

tensor([[17., 22., 27.],
        [22., 29., 36.],
        [27., 36., 45.]])

### Convert it to numpy array

In [39]:
import numpy as np
X.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [40]:
Y = np.array([[77, 88, 99], [10, 11, 12]])
Y

array([[77, 88, 99],
       [10, 11, 12]])

In [41]:
torch.tensor(Y) # default float presision is float64

tensor([[77, 88, 99],
        [10, 11, 12]])

In [42]:
torch.FloatTensor(Y) # specify float32

tensor([[77., 88., 99.],
        [10., 11., 12.]])

### Inplace Operations

In [43]:
X.relu_()

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [44]:
X.sqrt_()

tensor([[1.0000, 1.4142, 1.7321],
        [2.0000, 2.2361, 2.4495]])

In [45]:
X.zero_()

tensor([[0., 0., 0.],
        [0., 0., 0.]])